# 리뷰 기반 영화 추천 모델 생성하기 
- 영화 평가 데이터 셋을 [여기서 ](https://www.kaggle.com/datasets/grouplens/movielens-20m-dataset) 다운 받아주세요. 
- 현재 위치에 data 디렉토리를 만든 후에, 다운 받은 데이터를 data 안에 넣어주세요!

- 이 코드는 [블로그 ](https://velog.io/@kms1003/%EC%B6%94%EC%B2%9C%EC%8B%9C%EC%8A%A4%ED%85%9C-02.-%ED%98%91%EC%97%85-%ED%95%84%ED%84%B0%EB%A7%81-Collaborative-Filtering-1-%EC%82%AC%EC%9A%A9%EC%9E%90-%EA%B8%B0%EB%B0%98-CF-%EC%95%84%EC%9D%B4%ED%85%9C-%EA%B8%B0%EB%B0%98-CF#span-stylecolorolivedrab%EC%95%84%EC%9D%B4%ED%85%9C-%EA%B8%B0%EB%B0%98-cf-%EC%95%8C%EA%B3%A0%EB%A6%AC%EC%A6%98span) 를 참고합니다. 

In [82]:
import pandas as pd 
import numpy as np
from sklearn.metrics.pairwise  import cosine_similarity

## 0. 데이터 전처리

In [83]:
movie, rating = pd.read_csv('./data/movie.csv'), pd.read_csv('./data/rating.csv')

In [114]:
print(movie.head())
movie.info()

   movieId                               title  \
0        1                    Toy Story (1995)   
1        2                      Jumanji (1995)   
2        3             Grumpier Old Men (1995)   
3        4            Waiting to Exhale (1995)   
4        5  Father of the Bride Part II (1995)   

                                        genres  
0  Adventure|Animation|Children|Comedy|Fantasy  
1                   Adventure|Children|Fantasy  
2                               Comedy|Romance  
3                         Comedy|Drama|Romance  
4                                       Comedy  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27278 entries, 0 to 27277
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  27278 non-null  int64 
 1   title    27278 non-null  object
 2   genres   27278 non-null  object
dtypes: int64(1), object(2)
memory usage: 639.5+ KB


In [115]:
print(rating.head())
rating.info()

   userId  movieId  rating
0       1        2     3.5
1       1       29     3.5
2       1       32     3.5
3       1       47     3.5
4       1       50     3.5
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 900 entries, 0 to 899
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   userId   900 non-null    int64  
 1   movieId  900 non-null    int64  
 2   rating   900 non-null    float64
dtypes: float64(1), int64(2)
memory usage: 21.2 KB


In [116]:
# mtrx 만들기 
rating = rating[["userId", "movieId" , "rating"]][:900] #900명 데이터만 사용
mtrx = rating.pivot_table(index = 'userId', columns = 'movieId', values = 'rating')
mtrx.head()


movieId,1,2,3,6,7,10,11,15,16,17,...,7454,7482,7757,8368,8482,8507,8636,8690,8961,31696
userId,,,,,,,,,,,,,,,,,,,,,
1,NaN,3.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4.0,3.0,4.0,4.0,3.5,5.0,4.5,3.5,4.0,4.0
2,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,3.0,NaN,4.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,3.0,NaN,NaN,NaN,NaN,5.0,NaN,NaN,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [117]:
# 빈 값 0으로 채워줌 
rating_mtrx = mtrx.fillna(0)
rating_mtrx.head()

movieId,1,2,3,6,7,10,11,15,16,17,...,7454,7482,7757,8368,8482,8507,8636,8690,8961,31696
userId,,,,,,,,,,,,,,,,,,,,,
1,0.0,3.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,4.0,3.0,4.0,4.0,3.5,5.0,4.5,3.5,4.0,4.0
2,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,3.0,0.0,4.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,3.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## 1. 사용자 기반  Filtering 영화 간 유사도를 계산하기
사용자 ID 를 넣으면, 사용자 유사도 기반으로 추천할 만한 영화 제공하기 

-  사용할 데이터: user_id를 기준으로 row가 형성되어 있음 > 추후 userID를 기준으로 유사도 계산하기 위함이다. 

In [124]:
# 사용할 데이터: 
rating_mtrx.head()

movieId,1,2,3,6,7,10,11,15,16,17,...,7454,7482,7757,8368,8482,8507,8636,8690,8961,31696
userId,,,,,,,,,,,,,,,,,,,,,
1,0.0,3.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,4.0,3.0,4.0,4.0,3.5,5.0,4.5,3.5,4.0,4.0
2,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,3.0,0.0,4.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,3.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [118]:
# 유사도 계산 - 유사도 mtrx 만들기 
user_similarity = cosine_similarity(rating_mtrx, rating_mtrx) # 행과 행끼리 내적된다.. 주의 !
user_similarity_df = pd.DataFrame(user_similarity,
                               index = rating_mtrx.index,
                               columns = rating_mtrx.index)
user_similarity_df.head()

userId,1,2,3,4,5,6,7,8,9
userId,,,,,,,,,
1,1.000000,0.102916,0.261198,0.029091,0.139199,0.029488,0.112060,0.060641,0.063539
2,0.102916,1.000000,0.180124,0.064014,0.140042,0.092464,0.160006,0.103785,0.000000
3,0.261198,0.180124,1.000000,0.057323,0.156755,0.063552,0.207623,0.087128,0.071207
4,0.029091,0.064014,0.057323,1.000000,0.300828,0.040654,0.073242,0.307392,0.065407
5,0.139199,0.140042,0.156755,0.300828,1.000000,0.234223,0.169685,0.304162,0.000000


In [119]:
def get_recom_user(user_id, topN, df): # {user_id}를 입력하면, 해당 사람과 유사한 {num_of_people} 명의 사람을 기반으로 영화 추천 
    # 채우기 

    df = df.loc[[user_id], :] # 해당 user 행 선택 
    df = df.transpose() # 전치 

    df = df.sort_values(by=user_id) # 유사도 정렬
    topN_neighborsdf = df[:topN].index.values # 유사한 N 명의 아이디 저장 
    #print(topN_neighbors)
    
    # 각 사람에 대해서, 점수가 젤 높은 영화 찾기 
    to_recommend = []
    for neighbor in topN_neighbors:
        movie_id = int(rating_mtrx.loc[neighbor].sort_values(ascending=False).index[0])
        to_recommend.append(movie[movie['movieId'] == movie_id].title.values[0]) #영화 이름으로 바꿈. 

    return to_recommend

In [120]:
# 결과 실행 

result = get_recom_user(user_id = 1, topN = 4, df = user_similarity_df) # 몇 명까지 
print("추천할 영화 리스트 순위")
print('\n'.join(map(str,result)))

추천할 영화 리스트 순위
Lord of the Rings: The Two Towers, The (2002)
Godfather, The (1972)
Snow White and the Seven Dwarfs (1937)
Duck Soup (1933)


## 2. 아이템 기반  Filtering 영화 간 유사도를 계산하기
: 사용자가 본 영화를 기반으로, 새로운 N개의 아이템을 추천해보자!

In [121]:
# movie id 가 index가 되도록 행렬 전치 
rating_mtrx_T = rating_mtrx.transpose() #.copy()
rating_mtrx_T.head()

userId,1,2,3,4,5,6,7,8,9
movieId,,,,,,,,,
1,0.0,0.0,4.0,0.0,0.0,5.0,0.0,4.0,0.0
2,3.5,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0
3,0.0,4.0,0.0,0.0,0.0,3.0,3.0,5.0,0.0
6,0.0,0.0,0.0,3.0,0.0,0.0,0.0,3.0,0.0
7,0.0,0.0,0.0,0.0,0.0,5.0,3.0,0.0,0.0


In [122]:
# 영화 간 코사인 유사도 계산 
item_similarity = cosine_similarity(rating_mtrx_T, rating_mtrx_T)
item_similarity_df = pd.DataFrame(item_similarity,
                               index = rating_mtrx_T.index,
                               columns = rating_mtrx_T.index)
item_similarity_df.head()

movieId,1,2,3,6,7,10,11,15,16,17,...,7454,7482,7757,8368,8482,8507,8636,8690,8961,31696
movieId,,,,,,,,,,,,,,,,,,,,,
1,1.000000,0.0,0.603538,0.374634,0.567889,0.374634,0.000000,0.000000,0.000000,0.537169,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.000000,1.0,0.000000,0.000000,0.000000,0.000000,0.508183,0.000000,0.000000,0.316717,...,0.759257,0.759257,0.759257,0.759257,0.759257,0.759257,0.759257,0.759257,0.759257,0.759257
3,0.603538,0.0,1.000000,0.460287,0.535853,0.460287,0.243985,0.390567,0.390567,0.443508,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,0.374634,0.0,0.460287,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,0.567889,0.0,0.535853,0.000000,1.000000,0.000000,0.321403,0.514496,0.514496,0.862443,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [123]:
# 사용자가 본 영화의 id를 입력하면, {num_of_items}개의 영화를 추천해주는 함수 
def get_recom_item(movie_id, num_of_items):
    result =  item_similarity_df[movie_id].sort_values(ascending=False)[:11]

    # movie id를 영화 이름으로 바꾸기 
    for movie_id, value in result.items(): #result는 series 클래스 객체
        result[movie_id] = movie[movie['movieId'] == movie_id].title.values[0]
    return result[1:] # 자기 자신은 제외하기

# 실행 
print("##### 결과 ###### ")
get_recom_item(movie_id=648, num_of_items=3) 

##### 결과 ###### 


/var/folders/m9/2160l_511v1f6f2dms12xsr00000gn/T/ipykernel_39399/2258982282.py:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Mission: Impossible (1996)' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  result[movie_id] = movie[movie['movieId'] == movie_id].title.values[0]


movieId
1                  Toy Story (1995)
140    Up Close and Personal (1996)
141            Birdcage, The (1996)
17     Sense and Sensibility (1995)
500           Mrs. Doubtfire (1993)
150                Apollo 13 (1995)
3           Grumpier Old Men (1995)
595     Beauty and the Beast (1991)
508             Philadelphia (1993)
588                  Aladdin (1992)
Name: 648, dtype: object